# Cellphone Coverage

conda env list

conda activate geoenv

In [3]:
import geopandas as gpd
import pandas as pd
from shapely import wkt
from shapely.geometry import Point
import requests, zipfile, io

In [4]:
rawDataRelativePath = './dataSources'
targetFile = 'cellTowers_sample.csv'
pathAndFile = "%s/%s" % (rawDataRelativePath, targetFile)
cellTowers = gpd.read_file(pathAndFile)
cellTowers = gpd.GeoDataFrame(cellTowers, geometry=gpd.points_from_xy(cellTowers['field_7'], cellTowers['field_8']), crs="EPSG:4326")
cellTowers.head()

,field_1,field_2,field_3,field_4,field_5,field_6,field_7,field_8,field_9,field_10,field_11,field_12,field_13,field_14,geometry
0,GSM,302,720,10000,53872,0,-114.1704,51.1122,7483,144,1,1343312859,1691870141,0,None
1,GSM,302,720,10100,55752,0,-113.9728,50.8802,18027,157,1,1343318865,1721917750,0,None
2,GSM,302,720,10100,55751,0,-113.9555,50.9061,3019,153,1,1343318865,1721917750,0,None
3,GSM,302,720,10100,59852,0,-113.9815,50.8945,6592,119,1,1343318894,1721917750,0,None
4,GSM,302,720,10000,50752,0,-114.2764,51.0879,4911,171,1,1343358077,1719655092,0,None
5,GSM,302,720,10000,50753,0,-114.3839,51.1263,27073,102,1,1343358077,1691868176,0,None
6,GSM,302,720,10000,57842,0,-114.2936,51.0846,7628,146,1,1343358082,1719655092,0,None
7,GSM,302,720,10000,50751,0,-114.2425,51.114,14203,117,1,1343358087,1719655090,0,None
8,GSM,302,720,10000,57843,0,-114.3321,51.0892,31456,113,1,1343358088,1719655122,0,None
9,GSM,302,720,10100,52571,0,-114.0905,51.0245,6856,88,1,1343358599,1717046225,0,None


In [11]:
canadaRectangular = pd.DataFrame(
    {
        "Country": ["Canada"],
        "Coordinates": [
            "POLYGON ((-80 52, -55 52, -55 45, -80 45, -80 52))"
        ],
    }
)
canadaRectangular["Coordinates"] = gpd.GeoSeries.from_wkt(canadaRectangular["Coordinates"])
canadaRectangular = gpd.GeoDataFrame(canadaRectangular, geometry="Coordinates", crs="EPSG:4269")

In [12]:
cellTowersCanada = gpd.sjoin(cellTowers, canadaRectangular)
cellTowersCanada

/var/folders/jw/twx63rq17tjbgssr92x34s080000gq/T/ipykernel_2546/3412510144.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4269

  cellTowersCanada = gpd.sjoin(cellTowers, canadaRectangular)


,field_1,field_2,field_3,field_4,field_5,field_6,field_7,field_8,field_9,field_10,field_11,field_12,field_13,field_14,geometry,index_right,Country
